Because of this error, have to use sqlalchemy : 
pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

In [6]:
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine
import pandas as pd

# Replace placeholders with your Snowflake credentials
engine = create_engine(URL(
    user='baopham',
    password='XXX',
    account='gz74928.west-europe.azure',
    warehouse='COMPUTE_BAO',
    database='WILDWEST_9',
    schema='PROCESSED'
))

query = """
    select *,'business' as type from BMSG9901
    union all
    select *, 'business' as type from BMSG9902
    union all
    select *, 'business' as type from BMSG9903
    union all
    select *, 'business' as type from BMSG9904
    union all
    select *, 'business' as type from BMSG0001
    union all
    select *,'residental' as type from RMSG9906
"""
df = pd.read_sql(query, engine)

In [7]:
#concat area code, exchange and line to create telephone number
df["bill_num"] = df["bill_area_code"] + df["bill_exchange"] + df["bill_line"]
df = df.drop(['bill_area_code', 'bill_exchange','bill_line'], axis=1)

df["org_num"] = df["orig_area_code"] + df["orig_exchange"] + df["orig_line"]
df = df.drop(['orig_area_code', 'orig_exchange','orig_line'], axis=1)

df["term_num"] = df["term_area_code"] + df["term_exchange"] + df["term_line"]
df = df.drop(['term_area_code', 'term_exchange','term_line'], axis=1)



In [8]:
df.loc[df['con_hour'] > '23', 'con_hour'] = '23'
df.loc[df['con_min'] > '59', 'con_min'] = '59'
df.loc[df['con_sec'] > '59', 'con_sec'] = '59'
df['con_timestamp'] = pd.to_datetime(df['con_date'] + ' ' + df[['con_hour', 'con_min', 'con_sec']].astype(str).agg(':'.join, axis=1),format='%y%m%d %H:%M:%S')
df = df.drop(['con_date', 'con_hour','con_min', 'con_sec'], axis=1)
df.iloc[:,[0,11,1,2,7,8,9,10,6,3,4,5]]

,con_cst,con_timestamp,term_st,term_cntry,type,bill_num,org_num,term_num,local_sw_id,rev_amt,rev_min,rev_sec
0,12,1999-01-12 13:30:38,ME,,business,8084310210,8084310210,2078760210,73,013.85,14,20
1,16,1999-01-01 18:05:22,AZ,,business,2135510097,2135510097,2506780097,74,006.12,30,51
2,15,1999-01-01 17:53:43,WV,,business,4085390342,4085390342,3047530342,46,008.33,59,51
3,12,1999-01-12 13:13:12,ME,,business,5093510378,5093510378,2078900378,36,013.85,14,06
4,06,1999-01-01 08:08:09,,DE,business,4156810290,4156810290,9792580290,45,007.06,91,49
...,...,...,...,...,...,...,...,...,...,...,...,...
300001,11,1999-05-03 10:42:18,CA,,residental,9073210390,9073210390,8057490390,18,015.15,23,14
300002,07,1999-04-01 09:35:58,NJ,,residental,8016540231,8016540231,6096510231,37,006.06,24,27
300003,09,1999-05-19 07:42:38,WV,,residental,2136420328,2136420328,3044710328,73,009.90,11,49
300004,22,1999-06-01 00:56:28,VT,,residental,8016540046,8016540046,8026510046,03,009.09,76,05


In [9]:
df['rev_total_sec'] = df['rev_min']*60 + df['rev_sec']
df = df.drop(['rev_min', 'rev_sec'], axis = 1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300006 entries, 0 to 300005
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   con_cst        300006 non-null  object        
 1   term_st        300006 non-null  object        
 2   term_cntry     300006 non-null  object        
 3   rev_amt        300006 non-null  object        
 4   local_sw_id    300006 non-null  object        
 5   type           300006 non-null  object        
 6   bill_num       300006 non-null  object        
 7   org_num        300006 non-null  object        
 8   term_num       300006 non-null  object        
 9   con_timestamp  300006 non-null  datetime64[ns]
 10  rev_total_sec  300006 non-null  object        
dtypes: datetime64[ns](1), object(10)
memory usage: 25.2+ MB
